In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import os
from os import listdir
from os.path import isfile, join
import gdal
import ee
import ssl
import time
from pathlib import Path
import numpy as np
np.set_printoptions(threshold=np.inf)

In [ ]:

def load_clean_yield_data(yield_data_filepath):
    important_columns=['Year', 'State ANSI', 'County ANSI', 'Value']
    yield_data=pd.read_csv(yield_data_filepath).dropna(subset=important_columns,how='any')
    return yield_data

def get_tif_files(image_path):
    files = []
    for dir_file in image_path.iterdir():
        # print("dir_file")
        # print(str(dir_file))
        if str(dir_file).endswith('tif'):
            # strip out the directory so its just the filename
            files.append(str(dir_file.parts[-1]))
            # print(str(dir_file.parts[-1]))
    return files

def get_tif_files_12(image_path):#name length less than 12
    files = []
    for dir_file in image_path.iterdir():
        # print(str(dir_file))
        if str(dir_file).endswith('tif'):
            # strip out the directory so its just the filename
            if len(str(dir_file.parts[-1]))<12:#max possible name is ss_ccc.tif, len=10
                files.append(str(dir_file.parts[-1]))
            # print(str(dir_file.parts[-1]))
    return files

def get_tif_files_12(mask_path,temperature_path,image_path,weather_path):
    print(len([1 for x in list(os.scandir(mask_path)) if x.is_file()]))
    print(len([1 for x in list(os.scandir(temperature_path)) if x.is_file()]))
    print(len([1 for x in list(os.scandir(image_path)) if x.is_file()]))
    print(len([1 for x in list(os.scandir(weather_path)) if x.is_file()]))
    mask_files=[f for f in listdir(mask_path) if isfile(join(mask_path, f))]
    print("\n\n")
    print(mask_files)
    temperature_files=[f for f in listdir(temperature_path) if isfile(join(temperature_path, f))]
    print("\n\n")
    print(temperature_files)
    image_files=[f for f in listdir(image_path) if isfile(join(image_path, f))]
    print("\n\n")
    print(image_files)
    weather_files=[f for f in listdir(weather_path) if isfile(join(weather_path, f))]
    print("\n\n")
    print(weather_files)
    files = []
    for dir_file in image_path.iterdir():
        # print("dir_file")
        # print(str(dir_file))
        if str(dir_file).endswith('tif'):
            # strip out the directory so its just the filename
            if len(str(dir_file.parts[-1]))<12:#max possible name is ss_ccc.tif, len=10
                if str(dir_file.parts[-1]) in temperature_files:
                    if str(dir_file.parts[-1]) in weather_files:
                        if str(dir_file.parts[-1]) in mask_files:
                            files.append(str(dir_file.parts[-1]))
            # print(str(dir_file.parts[-1]))
    return files


In [ ]:
data = load_clean_yield_data('/content/yield_data.csv')
print('Total counties in missisippi', len(data[data['State ANSI']==28]['County ANSI'].unique()))
data[data['State ANSI']==28]['County ANSI'].unique()

Total counties in missisippi 53


array([ 51.,  89.,  97., 121., 123.,  17.,  87.,  95., 103.,  53.,  55.,
        83., 125., 133., 151., 163.,   9.,  13.,  33.,  43.,  71.,  93.,
       107., 137., 161.,   3.,  57.,  81., 115., 117., 139., 141., 145.,
        31., 129., 147.,   1.,  49., 149.,  11.,  27., 119., 135., 143.,
         7.,  15., 155.,  25., 101.,  63.,  21.,  77., 157.])

In [ ]:
# MAJOR_STATES=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16, 17, 18, 19, 20, 26, 27, 28, 29, 31, 38, 39, 46, 47, 55]
# MAJOR_STATES=[1, 5, 17, 18, 19, 20, 27, 29, 31, 38, 39, 46]
MAJOR_STATES=[17]

In [ ]:
try:
    # https://developers.google.com/earth-engine/guides/python_install-colab
    # Trigger the authentication flow.
    # Google Earth Engine Python Authenticator
    ee.Authenticate()
    # Initialize the Earth Engine module.
    ee.Initialize()
    print('Earth Engine initialized successfully!!')
except ee.EEException:
    print('Earth Engine failed to initialize. Use authenticate in command line.')

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=pESJxH4I6-MWhJlhhCvqdBMQzFXjyWjJAggGjEgQjUc&tc=1ATHeZfQIYLDUyMMvy2NqRY3inIXdOMJgVwyWEgOqI4&cc=1-HBiU2-MKXEg2lsTsaIkvlNcE6yzNME5FMMQKe-9-E

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWgavdenrtjnlmwUQCiuBYecOX9pAkuC3w5rI1MvO1YU2iyyr7Ii7nRjF9c

Successfully saved authorization token.
Earth Engine initialized successfully!!


In [ ]:
boundry =  ee.Geometry.Rectangle(-92, 43, -87, 36) # for 28 - missisippi
# boundry = ee.Geometry.Rectangle(-106.5, 50, -64, 23)
imgcoll = ee.ImageCollection('MODIS/MOD09A1').filterBounds(boundry).filterDate('2002-12-31', '2012-12-31') 

In [ ]:
imgcoll.first()

In [ ]:
!pip install rasterio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 20.9 MB 1.2 MB/s 


In [ ]:
from torchvision.transforms import ToTensor
from osgeo import gdal
ds = gdal.Open('/content/drive/MyDrive/crop_yield_image/31_11.tif')

myarray = np.array(ds.ReadAsArray())

AttributeError: ignored

In [ ]:
myarray.shape

In [ ]:
import rasterio
from rasterio.plot import show
fp = r'/content/drive/MyDrive/crop_yield_image/31_11.tif'
img = rasterio.open(fp)
show(img)

In [ ]:
img.read(2)[0]

In [ ]:
show(img.read(2000))

In [ ]:
dataset = img
dataset.count, dataset.height, dataset.width

In [ ]:
from matplotlib import pyplot as plt
img = image[20]

plt.imshow( img.numpy())

In [ ]:
class ModisExporter:
    """A class to export MODIS data from 
    Google Earth Engine to Google Drive

    Parameters"""

    def __init__(self, locations_filepath=Path('SOP_download/yield_data.csv'),collection_id='MODIS/006/MOD09A1'):#006 instead of 051
        self.locations=load_clean_yield_data(locations_filepath)
        self.collection_id=collection_id
        
    def update_parameters(self, location_filepath=None,collection_id=None):
        if location_filepath is not None:
            self.locations=load_clean_yield_data(location_filepath)
        if collection_id is not None:
            self.collection_id=collection_id

    @staticmethod
    def _export_one_image(img,folder, name, region, scale, crs):
        # export one image from Earth Engine to Google Drive
        # Author: Jiaxuan You, https://github.com/JiaxuanYou
        print(f'Exporting to {folder}/{name}')
        task_dict={
            'folder': folder,
            'fileNamePrefix': name,
            'scale': scale,
            'crs': crs
        }
        if region is not None:
            task_dict.update({
                'region': region
            })
        task=ee.batch.Export.image.toDrive(img,name, **task_dict)
        task.start()
        while task.status()['state']=='RUNNING':
            print('Running....')
            # perhaps task.cancel() at some point
            time.sleep(10)
        print(f'Done: {task.status()}')
    
    def export(self, folder_name, data_type, coordinate_system='EPSG:4326', scale=500,
               export_limit=10, min_img_val=None, max_img_val=None, major_states_only=True,
               check_if_done=False, download_folder=None):
        """Export an Image Collection from Earth Engine to Google Drive
        Parameters
        ----------
            folder_name: str
                The name of the folder to export the images to in
                Google Drive. If the folder is not there, this process
                creates it
            data_type: str {'image', 'mask', 'temperature'}
                The type of data we are collecting. This tells us which bands to collect.
            coordinate_system: str, default='EPSG:4326'
                The coordinate system in which to export the data
            scale: int, default=500
                The pixel resolution, as determined by the output.
                https://developers.google.com/earth-engine/scale
            export_limit: int or None, default=None
                If not none, limits the number of files exported to the value
                passed.
            min_img_val = int or None:
                A minimum value to clip the band values to
            max_img_val: int or None
                A maximum value to clip the band values to
            major_states_only: boolean, default=True
                Whether to only use the 11 states responsible for 75 % of national soybean
                production, as is done in the paper
            check_if_done: boolean, default=False
                If true, will check download_folder for any .tif files which have already been
                downloaded, and won't export them again. This effectively allows for
                checkpointing, and prevents all files from having to be downloaded at once.
            download_folder: None or pathlib Path, default=None
                Which folder to check for downloaded files, if check_if_done=True. If None, looks
                in data/folder_name
        """
        if check_if_done:
            if download_folder is None:
                # download_folder=Path('data')/folder_name    #DATA folder???
                download_folder=Path('')/folder_name
                already_downloaded= get_tif_files(download_folder)

        boundry =  ee.Geometry.Rectangle(-92, 43, -87, 36) # for 28 - missisippi
        # boundry = ee.Geometry.Rectangle(-106.5, 50, -64, 23)
        imgcoll = ee.ImageCollection(self.collection_id).filterBounds(boundry).filterDate('2002-12-31', '2012-12-31').sort('system:time_start', True)

        datatype_to_func={
            'image': _append_im_band,
            'mask': _append_mask_band,
            'temperature': _append_temp_band,
            'weather': _append_weather_band
        }

        img = imgcoll.iterate(datatype_to_func[data_type]) # data_type-> whether it is image, mask or temperature
        # stacking images all over the years for particular state and county
        img=ee.Image(img)   # need to explicitly use ee.Image because .iterate function returns Object

        # 'clip' the values of the band
        if min_img_val is not None:
            # passing an ee.Number creates a constant image
            img_min=ee.Image(ee.Number(min_img_val))
            img=img.min(img_min)

        if max_img_val is not None:
            # passing an ee.Number creates a constant image
            img_max=ee.Image(ee.Number(max_img_val))
            img=img.max(img_max)

        region = ee.FeatureCollection('TIGER/2018/Counties').filter(ee.Filter.eq('STATEFP', '17'))

        # https://developers.google.com/earth-engine/datasets/catalog/TIGER_2018_Counties
        def state_to_int(feature):
            return feature.set('COUNTYFP', ee.Number.parse(feature.get('COUNTYFP')))
        region=region.map(state_to_int)

        count=0
        for state_id, county_id in np.unique(self.locations[['State ANSI','County ANSI']].values,axis=0):
            if major_states_only:
                if (int)(state_id) not in MAJOR_STATES:
                    print(f'Skipping state id {int(state_id)}')
                    continue
            fname='{}_{}'.format(int(state_id),int(county_id))

            if check_if_done:
                if f'{fname}.tif' in already_downloaded:
                    print(f'{fname}.tif already downloaded! Skipping')
                    continue
            
            # getting the county for that particular state
            file_region = region.filterMetadata('COUNTYFP','equals', int(county_id))
            file_region = ee.Feature(file_region.first())
            processed_img = img.clip(file_region)
            file_region=None
            self._export_one_image(processed_img,folder_name,fname, file_region, scale, coordinate_system)
            count+=1
            if export_limit:
                if count>=export_limit:
                    print('Reached export limit!! Stopping...')
                    break
        print(f'Finished Exporting {count} files!')


    def export_all(self,export_limit=None, major_states_only=True, check_if_done=True,
                   download_folder=None):
        """
        Export all the data.
        download_folder = list of 3 pathlib Paths, for each of the 3 downloads
        """
        if download_folder is None:
            download_folder=[None]*4
        assert len(download_folder)==4, "Must have 3 download folders for the 3 exports!"

        #first, make sure the class was initialized correctly
        # self.update_parameters(locations_filepath=load_clean_yield_data('yield_data.csv'),
        #                        collection_id='MODIS/MOD09A1')        
        # self.export(folder_name='crop_yield_image', data_type='image',
        #             min_img_val=16000, max_img_val=100,
        #             export_limit=export_limit, major_states_only=major_states_only,
        #             check_if_done=check_if_done, download_folder=download_folder[0])

        # self.update_parameters(collection_id='MODIS/006/MCD12Q1')
        # self.export(folder_name='crop_yield_mask', data_type='mask',
        #             export_limit=export_limit, major_states_only=major_states_only,
        #             check_if_done=check_if_done, download_folder=download_folder[1])

        # self.update_parameters(collection_id='MODIS/006/MOD11A2')
        # self.export(folder_name='crop_yield_temp', data_type='temperature',
        #             export_limit=export_limit, major_states_only=major_states_only,
        #             check_if_done=check_if_done, download_folder=download_folder[2])
        
        self.update_parameters(collection_id="NASA/ORNL/DAYMET_V3")
        self.export(folder_name='crop_yield_daymet', data_type='weather',
                    export_limit=export_limit, major_states_only=major_states_only,
                    check_if_done=check_if_done, download_folder=download_folder[3])

        print('Done exporting! Download the folders from your Google Drive')


def _append_im_band(current,previous):
    # Transforms an Image Collection with 1 band per Image into a single Image with items as bands
    # Author: Jamie Vleeshouwer

    previous=ee.Image(previous)
    current=current.select([0, 1, 2, 3, 4, 5, 6])
    # Append it to the result (Note: only return current item on first element/iteration)
    return ee.Algorithms.If(ee.Algorithms.IsEqual(previous,None),current,previous.addBands(ee.Image(current)))
    # so basically we are adding bands here, and now our final image have all images and their corresponding bands 

def _append_mask_band(current,previous):
    # Transforms an Image Collection with 1 band per Image into a single Image with items as bands
    # Author: Jamie Vleeshouwer

    previous=ee.Image(previous)
    current=current.select([0])
    return ee.Algorithms.If(ee.Algorithms.IsEqual(previous,None),current,previous.addBands(ee.Image(current)))

def _append_temp_band(current,previous):
    # Transforms an Image Collection with 1 band per Image into a single Image with items as bands
    # Author: Jamie Vleeshouwer

    # Rename the band
    previous=ee.Image(previous)
    current=current.select([0,4])
    return ee.Algorithms.If(ee.Algorithms.IsEqual(previous, None), current, previous.addBands(ee.Image(current)))

def _append_weather_band(current,previous):
    # Transforms an Image Collection with 1 band per Image into a single Image with items as bands
    # Author: Jamie Vleeshouwer

    # Rename the band
    previous=ee.Image(previous)
    current=current.select([1,6])
    return ee.Algorithms.If(ee.Algorithms.IsEqual(previous, None), current, previous.addBands(ee.Image(current)))




In [ ]:
yield_data_path='yield_data.csv' #taken from pycrop
                                                                
yield_data_path = Path(yield_data_path)
exporter = ModisExporter(locations_filepath=yield_data_path)

In [ ]:
exporter.export_all(check_if_done=False)

Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1
Skipping state id 1


In [ ]:
image=np.transpose(np.array(gdal.Open(str('/content/drive/MyDrive/crop_yield_image/1_1.tif')).ReadAsArray(), dtype='uint16' ),
                        axes=(1,2,0))

In [ ]:
print(np.count_nonzero(image==12))
print(image.shape)

In [ ]:
var dataset = ee.FeatureCollection('TIGER/2018/Counties')
// .filter(ee.Filter.eq('STATEFP', '28'));
var visParams = {
  palette: ['purple', 'blue', 'green', 'yellow', 'orange', 'red'],
  min: 0,
  max: 50,
  opacity: 0.8,
};
// Turn the strings into numbers
dataset = dataset.map(function (f) {
  return f.set('COUNTYFP', ee.Number.parse(f.get('COUNTYFP')));
});
var file_region = dataset.filterMetadata('COUNTYFP','equals', 1);
var file_region = file_region.filter(ee.Filter.eq('STATEFP', '28'));
var file_region = ee.Feature(file_region.first());
print(file_region);
var image = ee.Image().float().paint(dataset, 'COUNTYFP');
var countyOutlines = ee.Image().float().paint({
  featureCollection: dataset,
  color: 'black',
  width: 1
});

// Map.setCenter(-99.844, 37.649, 5);
// Map.addLayer(image, visParams, 'TIGER/2018/Counties');
// Map.addLayer(countyOutlines, {}, 'county outlines');
// Map.addLayer(dataset, null, 'for Inspector', false);
// var polygon = ee.Geometry.Rectangle(-92, 36, -88, 29.5)
var polygon = file_region.geometry()
Map.centerObject(polygon);
Map.addLayer(polygon, {color: 'FF0000'});